<a href="https://colab.research.google.com/github/JotaBlanco/QuixStreamsNotebooks/blob/main/Workshops/Scandio/Quix_Streams_PUB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Quix Streams
Just use pip install to download the Quix Streams library.

[Quix Streams](https://github.com/quixio/quix-streams) is an open source Python library for processing streaming data. It’s aimed at people who work with time-series data streams — from developers and ML engineers to data scientists and data engineers.

In [1]:
! pip install quixstreams

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 16.4 MB/s eta 0:00:00


# Import the libraries
We will be using mainly pandas, quix, matplotlib and seaborn.

In [2]:
import pandas as pd
import quixstreams as qx

# 1 - Create client
Let's start by creating a Quix client that we'll use to publish and subscribe to Kafka topics.

In [3]:
# Initiating Quix managed token, but it could be your own kafka
token = 'sdk-296f2b9decff4770a525ff7d8855a78d'
client = qx.QuixStreamingClient(token)
client

# 2 - Producer client
To publish data to one topic, we will need to create a producer client pointing to that topic.

In [4]:
topic_name = "test-topic"
topic_producer = client.get_topic_producer(topic_name)
topic_producer

# 3 - Streams
Streams are ways to distribute the messages load into a topic very efficiently, allowing escalation whilst ensuring chronolofical order. Streams are to topics what road lines are to highways.
We don't need many streams yet, but let's see how they are created:

In [5]:
stream_id = "test-stream_1"
test_stream_1 = topic_producer.create_stream(stream_id)
test_stream_1

You can add properties to streams, like names and metadata.

In [6]:
test_stream_1.properties.name = "Tutorial Test Stream 1"
test_stream_1.properties.metadata["Test Number"] = "1"

# 4 - Data format
There are two data formats Quix Stream can use to publish data to topics:

## 4.1 TimeseriesData
TimeseriesData is the formal class in Quix Streams which represents a time series data packet in memory. The format consists of a list of Timestamps with their corresponding parameter names and values for each timestamp.

You should imagine a TimeseriesData as a table where the Timestamp is the first column of that table and where the parameters are the columns for the values of that table.

In [7]:
# This dataframe follows the proper TimeseriesData format: timestamp and different parameters
df = pd.DataFrame({
    "Timestamp": [pd.Timestamp.now(), pd.Timestamp.now() + pd.Timedelta("5sec")],
    "Param A": [10, 20],
    "Column B": [12, 9]
})
df

,Timestamp,Param A,Column B
0,2023-06-13 10:49:00.225477,10,12
1,2023-06-13 10:49:05.225550,20,9


In [8]:
# This is the actual way to define a qx.TimeseriesData object for that same data
timeseries_data = qx.TimeseriesData()
timeseries_data.add_timestamp(pd.Timestamp.now()) \
                .add_value("Param A", 10) \
                .add_value("Column B", 12)
timeseries_data.add_timestamp(pd.Timestamp.now() + pd.Timedelta("5sec")) \
                .add_value("Param A", 20) \
                .add_value("Column B", 9)

Conversions between qx.TimeseriesData and pd.Dataframe formats are easy:

In [9]:
timeseries_data.to_dataframe()

,timestamp,Param A,Column B
0,1686653385383974000,10.0,12.0
1,1686653390384630000,20.0,9.0


In [10]:
qx.TimeseriesData.from_panda_dataframe(df)

## 4.2 EventData
EventData consists of a record with a Timestamp, an EventId and an EventValue.

You should imagine a list of EventData instances as a simple table of three columns where the Timestamp is the first column of that table and the EventId and EventValue are the second and third columns, as shown in the following table.

In [11]:
event_data_1 = qx.EventData(
    event_id = "Door Open",
    time = pd.Timestamp.now(),
    value = "The front door of the house has just been open")
event_data_1

In [12]:
event_data_2 = qx.EventData(
    event_id = "Door Closed",
    time = pd.Timestamp.now(),
    value = "The front door of the house is back to closed state")
event_data_2

# 5 - Publish data to the topic
Let's publish each of the data messages created to one stream now:

## 5.1 TimeseriesData
Let's see how to publish the TimeseriesData object we created earlier.

In [13]:
timeseries_data

In [14]:
# Publishing it to stream 1
test_stream_1.timeseries.publish(timeseries_data)

In [15]:
# Also, pd.DataFrame objects can be published very simply
# (conversion to qx.TimeseriesData object is done automatically)
df

,Timestamp,Param A,Column B
0,2023-06-13 10:49:00.225477,10,12
1,2023-06-13 10:49:05.225550,20,9


In [16]:
# Publishing it to stream 1
test_stream_1.timeseries.publish(df)

## 5.2 EventData
Let's now publish the EventData messages we created earlier:

In [18]:
# Publishing event 1 and event 2 to stream 1
test_stream_1.events.publish(event_data_1)
test_stream_1.events.publish(event_data_2)